In [1]:
pip install varclushi

  Using cached varclushi-0.1.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached factor_analyzer-0.3.1-py3-none-any.whl.metadata (10 kB)
Using cached varclushi-0.1.0-py3-none-any.whl (6.3 kB)
Using cached factor_analyzer-0.3.1-py3-none-any.whl (41 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from varclushi import VarClusHi

In [16]:
demo1_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
demo1_df.drop('quality',axis=1,inplace=True)
demo1_vc = VarClusHi(demo1_df,maxeigval2=1,maxclus=None)
demo1_vc.varclus()

In [17]:
demo1_df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [18]:
demo1_vc.info

,Cluster,N_Vars,Eigval1,Eigval2,VarProp
0,0,3,2.141357,0.658413,0.713786
1,1,3,1.766885,0.900991,0.588962
2,2,2,1.371260,0.628740,0.685630
3,3,2,1.552496,0.447504,0.776248
4,4,1,1.000000,0.000000,1.000000


In [19]:
demo1_vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,fixed acidity,0.882210,0.277256,0.162976
1,0,density,0.622070,0.246194,0.501362
2,0,pH,0.637076,0.194359,0.450478
3,1,free sulfur dioxide,0.777796,0.010358,0.224530
4,1,total sulfur dioxide,0.786660,0.042294,0.222761
5,1,residual sugar,0.202428,0.045424,0.835525
6,2,sulphates,0.685630,0.106022,0.351653
7,2,chlorides,0.685630,0.048903,0.330534
8,3,citric acid,0.776248,0.398208,0.371810
9,3,volatile acidity,0.776248,0.040920,0.233299


In [20]:
demo1_vc.rsquare.Cluster

0     0
1     0
2     0
3     1
4     1
5     1
6     2
7     2
8     3
9     3
10    4
Name: Cluster, dtype: int64

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, SparsePCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

class ClustVarLV:
    def __init__(self, n_clusters=3, n_components=1, max_clusters=10, init_partition=None, threshold=0.5, use_sparse_pca=False):
        """
        Initialise le clustering des variables latentes avec l'option de SparsePCA.
        :param n_clusters: nombre initial de clusters à former.
        :param n_components: nombre de composantes principales à utiliser pour chaque cluster.
        :param max_clusters: nombre maximum de clusters (pour k+1).
        :param init_partition: partition initiale des variables (optionnel, liste de labels de clusters pour chaque variable).
        :param threshold: seuil de qualité pour ajouter des clusters (k+1).
        :param use_sparse_pca: booléen pour utiliser SparsePCA à la place de la PCA classique.
        """
        self.n_clusters = n_clusters
        self.n_components = n_components
        self.max_clusters = max_clusters
        self.init_partition = init_partition
        self.threshold = threshold
        self.use_sparse_pca = use_sparse_pca
        self.scaler = StandardScaler()
        self.pca_models = {}
        self.cluster_model = None
        self.cluster_labels = None
    
    def fit(self, X):
        """
        Ajuste le modèle sur les données.
        :param X: DataFrame ou matrice des données (variables en colonnes).
        """
        # Standardisation des données
        X_scaled = self.scaler.fit_transform(X)
        X_transposed = X_scaled.T
        
        if self.init_partition is not None:
            # Utiliser la partition initiale si fournie
            print("Partition initiale fournie. Utilisation directe.")
            self.cluster_labels = np.array(self.init_partition)
        else:
            # Appliquer k+1 si la qualité des clusters n'est pas satisfaisante
            for k in range(self.n_clusters, self.max_clusters + 1):
                self.cluster_model = KMeans(n_clusters=k, random_state=0)
                # Ajuster le modèle de clustering
                self.cluster_labels = self.cluster_model.fit_predict(X_transposed)
                
                # Calculer la qualité du clustering avec le score de silhouette
                score = silhouette_score(X_transposed, self.cluster_labels)
                print(f'Nombre de clusters: {k}, Score de silhouette: {score:.3f}')
                
                # Si la qualité est suffisante, on s'arrête
                if score >= self.threshold:
                    print(f'Clustering arrêté avec {k} clusters.')
                    break
        
        # Ajuster une PCA ou SparsePCA pour chaque cluster de variables
        self.pca_models = {}
        for cluster in np.unique(self.cluster_labels):
            # Sélectionner les variables appartenant à ce cluster
            variables_in_cluster = X.iloc[:, self.cluster_labels == cluster]
            
            if self.use_sparse_pca:
                # Utiliser SparsePCA
                print(f"Utilisation de SparsePCA pour le cluster {cluster}.")
                pca = SparsePCA(n_components=self.n_components, random_state=0)
            else:
                # Utiliser la PCA classique
                pca = PCA(n_components=self.n_components)
                
            self.pca_models[cluster] = pca.fit(variables_in_cluster)
    
    def transform(self, X):
        """
        Transforme les données en variables latentes, une pour chaque cluster.
        :param X: DataFrame ou matrice des données (variables en colonnes).
        :return: DataFrame avec les variables latentes pour chaque cluster.
        """
        X_scaled = self.scaler.transform(X)
        X_transposed = X_scaled.T

        latent_vars = []
        for cluster in np.unique(self.cluster_labels):
            variables_in_cluster = X.iloc[:, self.cluster_labels == cluster]
            pca = self.pca_models[cluster]
            latent_var = pca.transform(variables_in_cluster)
            latent_vars.append(latent_var)
        
        # Fusionner les variables latentes en un DataFrame
        latent_vars = np.hstack(latent_vars)
        return pd.DataFrame(latent_vars, columns=[f'Cluster_{i}_Latent' for i in range(latent_vars.shape[1])])

    def fit_transform(self, X):
        """
        Ajuste le modèle et transforme les données.
        :param X: DataFrame ou matrice des données (variables en colonnes).
        :return: DataFrame avec les variables latentes pour chaque cluster.
        """
        self.fit(X)
        return self.transform(X)

In [2]:
# Exemple d'utilisation :
if __name__ == "__main__":
    # Exemple de données
    data = pd.DataFrame({
        'Var1': [1, 2, 3, 4, 5],
        'Var2': [2, 3, 4, 5, 6],
        'Var3': [10, 11, 10, 12, 11],
        'Var4': [20, 21, 19, 20, 22],
        'Var5': [3, 3, 4, 2, 1],
        'Var6': [1, 0, 1, 1, 0]
    })
    
    # Partition initiale des variables (labels des clusters)
    init_partition = [0, 0, 1, 1, 2, 2]
    
    # Créer l'objet ClustVarLV avec SparsePCA
    clust_var_lv = ClustVarLV(n_clusters=2, n_components=1, max_clusters=5, init_partition=init_partition, use_sparse_pca=True)
    
    # Ajuster et transformer les données
    latent_variables = clust_var_lv.fit_transform(data)
    
    # Afficher les variables latentes
    print(latent_variables)

Partition initiale fournie. Utilisation directe.
Utilisation de SparsePCA pour le cluster 0.
Utilisation de SparsePCA pour le cluster 1.
Utilisation de SparsePCA pour le cluster 2.
   Cluster_0_Latent  Cluster_1_Latent  Cluster_2_Latent
0          2.800423         -0.396040         -0.396040
1          1.400211          0.594059         -0.396040
2          0.000000         -1.386139         -1.386139
3         -1.400211         -0.396040          0.594059
4         -2.800423          1.584158          1.584158
